1. make new models 

    <pre>
     12 class Step(models.Model):
     13     title = models.CharField(max_length=255)
     14     description = models.TextField()
     15     order = models.IntegerField(default=0)
     16     course = models.ForeignKey(Course)
     17
     18     def __str__(self):
     19         return self.title
    </pre>

2. make new migration

    <pre>

    dockeruser@4fc6563981f2:~/hostname/workspace/git/kaden/dss/Practice/treehouse/learning_site$ python manage.py makemigrations courses
    Migrations for 'courses':
    courses/migrations/0002_step.py
     Create model Step
    dockeruser@4fc6563981f2:~/hostname/workspace/git/kaden/dss/Practice/treehouse/learning_site$ python manage.py migrate courses
    Operations to perform:
    Apply all migrations: courses
    Running migrations:
    Applying courses.0002_step... OK    
    dockeruser@4fc6563981f2:~/hostname/workspace/git/kaden/dss/Practice/treehouse/learning_site

    </pre>





3. apply the migration (register)


<pre>
     in /courses/admin.py


      1 from django.contrib import admin
      2
      3 from .models import Course
      4 # Register your models here.
      5
      6 admin.site.register(Course)
      7 admin.site.register(Step)
    ~                              
</pre>

Django lets us create smaller forms that represent another model inside the admin form for a model. 

make class for smaller form : inline (the other one is tabular inlines)

make courseadmin for handling our courses

<pre>
  1 from django.contrib import admin
  2
  3 from .models import Course, Step
  4
  5
  6 class StepInline(admin.StackedInline):
  7     model = Step
  8
  9
 10 class CourseAdmin(admin.ModelAdmin):
 11     inlines = [StepInline,]
 12
 13
 14 admin.site.register(Course, CourseAdmin)
 15 admin.site.register(Step)
 
 </pre>
 
 IntegerField is a field that holds integers, or whole numbers.

An inline is a smaller form inside of a larger form. The smaller form represents a related record in the database.

StackedInline is an inline where each field takes up the full width of the form. Fields are stacked.

TabularInline is an inline where each field is part of a single row for the form.

https://docs.djangoproject.com/en/1.8/ref/contrib/admin/#inlinemodeladmin-objects

# Add a detail view
admin, view, editor

1. change /courses/views.py (pk is primary key which later handled

    <pre>
     12 def course_detail(request, pk):
     13     course = Course.objects.get(pk=pk)
     14     return render(request, 'courses/course_detail.html', {'course': course})
     </pre>
 
2. change /courses/urls.py # don't need ^(shaveron)

    <pre>
      1 from django.conf.urls import url
      2 from . import views
      3
      4 urlpatterns = [
      5     url(r'^$', views.course_list),
      6     url(r'(?P<pk>\d+)/$', views.course_detail),
      7 ]
    ~       
    </pre>
    
3. template :

 in /courses/templates/courses/course_detail.html
 
    <pre>
       1 {% extends "layout.html" %}
      2
      3 {% block title %} {{ course.title }}{% endblock %}
      4
      5 {% block content %}
      6 <articlee>
      7     <h2>{{ course.title }}</h2>
      8     {{ course.description }}
      9
     10     <section>
     11         {% for step in coourse.step_set.all %}
     12             <h3>{{ step.title }}</h3>
     13             {{ step.description }}
     14         {% endfor %}
     15     </section>
     16 </article>
     17 {% endblock %}
     </pre>

# Ordering and 404

what if we need reordering of the Step?

class Meta:  # class inside a class that control how that model does a few things. 

at http://192.168.99.100:8000/admin/courses/course/1/change/
you can control the order of the Step

in /courses/views/py

<pre>

1 from django.shortcuts import get_object_or_404, render
  2
  3 from .models import Course
  4
  5
  6 # Create your views here.
  7 def course_list(request):
  8     courses = Course.objects.all()
  9     return render(request, 'courses/course_list.html', {'courses': courses})
 10
 11
 12 def course_detail(request, pk):
 13     course = get_object_or_404(Course, pk=pk)
 14     return render(request, 'courses/course_detail.html', {'course': course})
 
 </pre>

then we get when we type unavailable page on browser

Page not found (404)
Request Method:	GET
Request URL:	http://192.168.99.100:8000/courses/18/
Raised by:	courses.views.course_detail
No Course matches the given query.
You're seeing this error because you have DEBUG = True in your Django settings file. Change that to False, and Django will display a standard 404 page.

class Meta:
    ordering = ['field1', 'field2']
This will cause the model to be ordered by field1, then field2 if there are any conflicts on field1 (two instances having the same field1 value). Finally, they'll be sorted by id if a conflict still exists.

get_object_or_404(Model, [selectors]) - Gets an object of Model by using whatever selection arguments have been given. For example: get_object_or_404(User, username='kennethlove') would try to get a User with an username set to "kennethlove". If that User didn't exist, a 404 error would be raised.

What's the long way? Consider this view:

from django.http import Http404

from .models import Course

def course_detail(request, pk):
    try:
        course = Course.objects.get(pk=pk)
    except Course.DoesNotExist:
        raise Http404()
    else:
        return render(request, 'courses/course_detail.html', {'course': course})
It's definitely more work!

If you want, you can customize your error views.
https://docs.djangoproject.com/en/1.8/topics/http/views/#customizing-error-views